# Install & load model

In [238]:
%pip install transformers plotly tqdm numpy torch torchvision torchaudio pandas nbformat scipy imageio kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 MB 26.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [73]:
from transformers import BertModel, BertTokenizer
import plotly.express as px
from tqdm import tqdm
import glob
import numpy as np
import scipy
import torch
import random
import imageio
import plotly.graph_objects as go
import os
import pandas as pd
torch.no_grad()

model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# First view of the embeddings

In [168]:
text = "The cat hunts the mice, it will kill"
tokens = tokenizer.tokenize(text)
input_ids = tokenizer.convert_tokens_to_ids(tokens)
print('\n'.join([f"{k}: {v}" for k, v in dict(zip(tokens, input_ids)).items()]))

the: 1996
cat: 4937
hunts: 28526
mice: 12328
,: 1010
it: 2009
will: 2097
kill: 3102


In [169]:
input_tensor = torch.tensor(input_ids).unsqueeze(0)
outputs = model(input_tensor)
embeddings = outputs.last_hidden_state[0]
print(embeddings, '\n')
print(embeddings.shape)

tensor([[-0.4705,  0.3865,  0.0080,  ..., -0.0251,  0.8186,  0.6171],
        [-0.3898, -0.1316, -0.1344,  ..., -0.1768,  0.5231,  0.8272],
        [ 0.0273,  0.3467,  0.0025,  ...,  0.0073,  0.3787,  0.2330],
        ...,
        [-0.6356, -0.2228, -0.0437,  ..., -0.2903,  0.3883,  0.7363],
        [-0.1286, -0.1739, -0.0790,  ..., -0.0559,  0.5166,  0.5161],
        [ 0.3648,  0.1465,  0.3094,  ..., -0.2486,  0.4267,  0.0918]],
       grad_fn=<SelectBackward0>) 

torch.Size([9, 768])


In [176]:
outputs.pooler_output[0].shape

torch.Size([768])

In [ ]:
outputs = model(torch.tensor([4937]).unsqueeze(0))
cat_embed = outputs.last_hidden_state[0]
print(embeddings[1, :3])
print(cat_embed[:,:3])

# Get "raw" embeddings for each token

In [ ]:
vocab = list(tokenizer.get_vocab().keys())
print(vocab[10000:10010])
print("Length:", len(vocab))

In [ ]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("hello"))

In [ ]:
tokenizer.decode(torch.tensor([7592]))

In [ ]:
begin = 0
end = len(vocab)

vocab_emb = [model(torch.tensor([i]).unsqueeze(0)).last_hidden_state[0] for i in tqdm(range(begin, end))]
vocab_emb = torch.stack(vocab_emb).reshape(-1, 768).detach().numpy()
np.save(f"vocab_emb_matrix.npy", vocab_emb)
#np.save(f"vocab_emb_matrix_{begin}-{end}.npy", vocab_emb)

In [ ]:
# In case you want to concatenate the matrices from different runs

file_list = glob.glob("*.npy")
matrices = [np.load(file) for file in file_list]
vocab_emb_matrix = np.concatenate(matrices, axis=0)
np.save('vocab_emb_matrix.npy', vocab_emb_matrix)

In [ ]:
vocab_emb_matrix.shape

In [ ]:
fig = px.histogram(vocab_emb_matrix[:, 200])
fig.show()

Compute correlation and display it

In [ ]:
# Charger la matrice concaténée
vocab_emb_matrix = np.load('vocab_emb_matrix.npy')

# Calculer la moyenne et l'écart-type pour chaque colonne
means = np.mean(vocab_emb_matrix, axis=0)
std_devs = np.std(vocab_emb_matrix, axis=0)

# Normalisation : soustraire la moyenne et diviser par l'écart-type pour chaque colonne
vocab_emb_matrix = (vocab_emb_matrix - means) / std_devs

# Vérification
print("Moyennes:", np.mean(vocab_emb_matrix, axis=0)[:5])
print("Écarts-types:", np.std(vocab_emb_matrix, axis=0)[:5])

In [ ]:
# Calculer la matrice de corrélation
correlation_matrix = np.corrcoef(vocab_emb_matrix, rowvar=False)
np.save('correlation_matrix.npy', correlation_matrix)

# Print the shape
print('shape', correlation_matrix.shape)

# Display heatmap
fig = px.imshow(correlation_matrix)
fig.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go

def matrix_stats(matrix):
    """Retourne les statistiques de la matrice en excluant la diagonale."""
    values = matrix[~np.eye(matrix.shape[0],dtype=bool)]
    return {
        'min': np.min(values),
        'max': np.max(values),
        'mean': np.mean(values),
        'median': np.median(values)
    }

# Supposons que corr_matrix soit votre matrice de corrélation
# corr_matrix = np.corrcoef(data, rowvar=False)  # Exemple

stats = matrix_stats(correlation_matrix)
print("Statistics:", stats)

# Histogramme des valeurs de corrélation
fig2 = go.Figure(data=go.Histogram(x=correlation_matrix[~np.eye(correlation_matrix.shape[0], dtype=bool)], nbinsx=50))
fig2.update_layout(title='Distribution des Valeurs de Corrélation', xaxis_title='Valeur de Corrélation', yaxis_title='Compte')
fig2.show()


In [ ]:
fig = px.line(correlation_matrix[100])
fig.show()

In [ ]:
corr_matrix_no_diag = np.copy(correlation_matrix)
np.fill_diagonal(corr_matrix_no_diag, 0)
maxs = np.max(corr_matrix_no_diag, axis=1)
mins = np.min(corr_matrix_no_diag, axis=1)

# Création des histogrammes avec plotly.graph_objects
fig = go.Figure()
fig.add_trace(go.Histogram(x=maxs, name='Maxs', opacity=0.75))
fig.add_trace(go.Histogram(x=mins, name='Mins', opacity=0.75))

# Mise à jour du layout pour mieux visualiser les histogrammes superposés
fig.update_layout(barmode='overlay', title="Distribution des Maxs et Mins")
fig.show()

In [ ]:
coef_corr = 0.5
no_corr_dims = [dmax < coef_corr and dmin > -coef_corr for dmax, dmin in zip(maxs, mins)]
print(f"Nombre de dimensions sans corrélation: {sum(no_corr_dims)}/{len(no_corr_dims)}")
#print(f"Dimensions sans corrélation: {np.where(no_corr_dims)}")

print('Moyenne des correlations max pour les dim sans corr:', np.mean(maxs[no_corr_dims]))
print('Moyenne des correlations min pour les dim sans corr:', np.mean(mins[no_corr_dims]))

# Study absolute correlation 

In [ ]:
# Charger la matrice concaténée
correlation_matrix = np.load('correlation_matrix.npy')

# Calculer la matrice de corrélation absolue
abs_corr_matrix = np.abs(correlation_matrix)

# Display heatmap
fig = px.imshow(abs_corr_matrix)
fig.show()

# Histogramme des valeurs de corrélation absolue
fig2 = go.Figure(data=go.Histogram(x=abs_corr_matrix[~np.eye(correlation_matrix.shape[0], dtype=bool)], nbinsx=50))
fig2.update_layout(title='Distribution des Valeurs de Corrélation', xaxis_title='Valeur de Corrélation', yaxis_title='Compte')
fig2.show()

# Sort dimensions (absolute correlation)

In [45]:
def fitness(path, abs_corr_matrix):
    N = len(path)
    distance = 0
    for ia, dim_a in enumerate(path):
        for ib, dim_b in enumerate(path):
            ang_a = ia / N * 2 * np.pi
            ang_b = ib / N * 2 * np.pi
            ang_dist = np.pi - abs(np.pi - abs(ang_a - ang_b)) # Distance angulaire bornée par Pi
            expected_dist = max(np.pi - 2*np.pi*abs_corr_matrix[dim_a, dim_b], 0) # Distance angulaire attendue
            distance += (ang_dist - expected_dist) ** 2 # cout erreur quadratique
    return distance

def crossover(parent1, parent2):
    """Effectue un croisement en prenant un sous-chemin de parent1 et complète avec parent2."""
    # Sélectionne un sous-chemin aléatoire de parent1
    start, end = random.sample(range(len(parent1)), 2)
    
    # Crée un enfant en copiant le sous-chemin de parent1
    child = [-1]*len(parent1)
    if start <= end:
        for i in range(start, end + 1):
            child[i] = parent1[i]
    else:
        for i in range(start, len(parent1)):
            child[i] = parent1[i]
        for i in range(0, end + 1):
            child[i] = parent1[i]

    # Complète l'enfant avec les éléments de parent2
    pointer = 0
    for i in range(len(child)):
        if child[i] == -1:
            while parent2[pointer] in child:
                pointer += 1
            child[i] = parent2[pointer]
            pointer += 1

    return child

def mutate(path, mutation_rate, abs_corr_matrix, top_n):
    """Déplace quelques éléments du chemin à une autre position plus adaptée."""
    # Choose random elements to remove from the path
    selected_elems = [elem for elem in path if random.random() < mutation_rate]
    new_path = [elem for elem in path if elem not in selected_elems]

    # Insert the removed elements at random best position
    for elem in selected_elems:

        # Compute the score of each position
        scores = []
        for i in range(len(new_path)):
            prev = new_path[i]
            next = new_path[(i+1)%len(new_path)]
            scores += [abs_corr_matrix[prev, elem] + abs_corr_matrix[elem, next]]

        # Select the top_n best positions
        ranked_indices = np.argsort(scores)[-top_n:] # Index of the top_n positions
        ranked_scores = np.sort(scores)[-top_n:] # Score of the top_n positions

        # Normalize the scores to get a probability distribution
        proba = ranked_scores / np.sum(ranked_scores)

        # Choose a position according to the probability distribution
        new_pos = np.random.choice(ranked_indices, p=proba)
        new_path.insert(new_pos, elem)
    
    return new_path

def genetic_algorithm(abs_corr_matrix, population_size, generations, mutation_rate, top_n_mutate, top_n_parents):
    """Effectue l'algorithme génétique pour le TSP."""
    
    # Génère une population initiale de chemins aléatoires
    population = [list(np.random.permutation(len(abs_corr_matrix))) for _ in range(population_size)]
    best_distance = np.inf
    best_path = None

    for generation in range(generations):
        # Évalue la fitness de chaque individu
        distances = [fitness(path, abs_corr_matrix) for path in population]
        ranked_indices = np.argsort(distances)
        print(f"Gen {generation}: {min(distances)}")
        #print('scores', distances)

        # Sauvegarde le meilleur individu si meilleur que courant
        if min(distances) < best_distance:
            best_distance = min(distances)
            best_path = population[np.argmin(distances)]

        # Sélection des parents pour le croisement
        parents = [population[i] for i in ranked_indices[:top_n_parents]]
        #print('parents', parents)

        # Création de la nouvelle population
        new_population = [p for p in parents] # Copie les parents
        while len(new_population) < population_size:
            parent1, parent2 = random.sample(parents, 2)
            child = crossover(parent1, parent2)
            new_population.append(child)

        # Appliquer la mutation
        population = [mutate(path, mutation_rate, abs_corr_matrix, top_n_mutate) for path in new_population]

    # Renvoie le meilleur chemin trouvé
    return best_path, best_distance

path, distance = genetic_algorithm(
    # corr_matrix=correlation_matrix,
    abs_corr_matrix=abs_corr_matrix, 
    population_size=100, 
    generations=1500, 
    mutation_rate=0.01, 
    top_n_mutate=5, 
    top_n_parents=10,
)

print("Meilleur chemin:", path)
print("Distance:", distance)

with open('path.txt', 'w') as f:
    f.write(str(path) + '\n')
    f.write(str(distance))

Gen 0: 1085794.1356755034
Gen 1: 1085643.418171864
Gen 2: 1084824.2151979185
Gen 3: 1084348.0236226844
Gen 4: 1082817.056391608
Gen 5: 1082290.0322586985
Gen 6: 1081241.4408383835
Gen 7: 1080115.1656163265
Gen 8: 1079152.4288942986
Gen 9: 1077364.3173742716
Gen 10: 1075586.4845291423
Gen 11: 1073924.99164325
Gen 12: 1072395.5149253255
Gen 13: 1070902.653328526
Gen 14: 1068881.2657587673
Gen 15: 1067558.5982620735
Gen 16: 1065692.2082789359
Gen 17: 1063155.6839707985
Gen 18: 1061190.7020502056
Gen 19: 1059052.819226459
Gen 20: 1055963.476367694
Gen 21: 1054495.98836774
Gen 22: 1052445.2166018372
Gen 23: 1051306.6335522057
Gen 24: 1050050.2486386658
Gen 25: 1048384.9362980399
Gen 26: 1046300.5401706774
Gen 27: 1044422.3200015379
Gen 28: 1042190.1445735891
Gen 29: 1041364.3004994459
Gen 30: 1039118.6055399809
Gen 31: 1038350.9711677422
Gen 32: 1036989.1890940322
Gen 33: 1035016.3315702028
Gen 34: 1032540.8266377762
Gen 35: 1030987.4130681419
Gen 36: 1027438.5201953356
Gen 37: 1026255.4308

# Radar Chart (absolute correlation)

In [5]:
# Load path
with open('path.txt', 'r') as f:
    path = eval(f.readline())

# Load vocab_emb_matrix
vocab_emb_matrix = np.load('vocab_emb_matrix.npy')

# Load the correlation matrix
correlation_matrix = np.load('correlation_matrix.npy')

# Compute coef for each value in the path
coefs = [1] # List that contains values {1, -1} for each value in the path
current_coef = 1
for i in range(len(path)):
    current = path[i]
    next = path[(i+1)%len(path)]
    next_coef = int(np.sign(correlation_matrix[current, next]))
    coefs += [current_coef * next_coef]
    current_coef = next_coef

print(coefs)

[1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, 1, -1, -1, -1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, -1, -1, -1, -1, 1, -1, 1, -1, 1, -1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, -1, 1, 1, 1, 1, 1, -1, 1, -1, 1, -1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, -1, 1, 1, -1, -1, 1, -1, 1, -1, 1, -1, -1, -1, -1, 1, -1, 1, 1, -1, -1, 1, 1, 1, 1, -1, 1, -1, -1, -1, -1, 1, 1, -1, -1, 1, -1, -1, -1, 1, 1, -1, 1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, -1, -1, -1, 1, 1, -1, 1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, 1, -1, -1, 1, 1, 1, -1, 1, 1, 1, 1, -1, -1, -1, -1, 1, -1, -1, 1, -1, 1, 1, 1, -1, -1, -1, 1, -1, -1, -1, 1, 1, 1, -1, 1, -1, 1, 1, -1, 1, 1, 1, -1, 1, -1, 1, -1, 1, 1, -1, -1, 1, 1, -1, -1, -1, 1, -1, -1, -1, -1, 1, 1, 1, -1, 1, -1, 1, -1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, 1, 1, -1, 1, -1, -1, -1, 1, -1, 1, 1, 1, 1, -1, 1, -1, 1, -1, -1, 1, -1, 1, 1, 1, 1, -1, -1, -1, -1, -1, 1, -1, 1, -1, -1, -1, 1, 1, -1, -1, -1

##### Cosine similarity

In [79]:
# Compute cosine similary 

def cosine_similarity(A, B):
    dot_product = np.dot(A, B)
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)
    similarity = dot_product / (norm_A * norm_B)
    return similarity

def compute_word_representation(word, vocab_emb_matrix, path, coefs, filter):
    """Compute the word representation after applying coefficient and convolution filter."""
    id = tokenizer.convert_tokens_to_ids([word])[0]
    print(str(tokenizer.decode([id])), end=' ')
    word_repr = np.array([vocab_emb_matrix[id, dim] * coefs[i] for i, dim in enumerate(path)])
    word_conv = circular_convolution(word_repr, filter)
    return word_repr, word_conv

def compute_similarity(w1, w2, vocab_emb_matrix, path, coefs, filter):
    """Compute the cosine similarity before and after for each pair."""
    w1b, w1a = compute_word_representation(w1, vocab_emb_matrix, path, coefs, filter)
    w2b, w2a = compute_word_representation(w2, vocab_emb_matrix, path, coefs, filter)
    return cosine_similarity(w1b, w2b), cosine_similarity(w1a, w2a)

close_pairs = [
    ("joy", "happiness"),
    ("angry", "furious"),
    ("laugh", "giggle"),
    ("jump", "leap"),
    ("run", "sprint"),
    ("fear", "terror"),
    ("sad", "unhappy"),
    ("build", "construct"),
    ("breeze", "wind"),
    ("chilly", "cold"),
    ("tired", "exhausted"),
    ("story", "tale"),
    ("smile", "grin"),
    ("car", "automobile"),
    ("house", "home"),
    ("picture", "photograph"),
    ("child", "kid"),
    ("idea", "thought"),
    ("funny", "humorous"),
    ("quick", "rapid"),
    ("ocean", "sea"),
    ("weather", "climate"),
    ("mountain", "peak"),
    ("boat", "ship"),
    ("heart", "cardio"),
    ("mind", "intellect"),
    ("snake", "serpent"),
    ("flower", "blossom"),
    ("fire", "flame"),
    ("bird", "avian"),
    ("star", "celestial"),
    ("river", "stream"),
    ("forest", "woods"),
    ("money", "currency"),
    ("glass", "crystal"),
    ("stone", "rock"),
    ("rain", "drizzle"),
    ("thought", "idea"),
    ("danger", "peril"),
    ("freedom", "liberty")
]

distant_pairs = [
    ("cat", "cement"),
    ("book", "apple"),
    ("moon", "chair"),
    ("water", "desert"),
    ("sun", "ice"),
    ("fish", "rocket"),
    ("music", "mountain"),
    ("computer", "river"),
    ("love", "equation"),
    ("light", "sadness"),
    ("art", "frog"),
    ("poetry", "insect"),
    ("happiness", "garbage"),
    ("science", "feather"),
    ("philosophy", "basketball"),
    ("thunder", "sorrow"),
    ("novel", "cactus"),
    ("ocean", "lantern"),
    ("painting", "hunger"),
    ("dream", "ladder"),
    ("galaxy", "ant"),
    ("pyramid", "novel"),
    ("keyboard", "volcano"),
    ("butterfly", "library"),
    ("castle", "equation"),
    ("symphony", "athlete"),
    ("rainforest", "algorithm"),
    ("theater", "molecule"),
    ("sculpture", "astronaut"),
    ("laptop", "meadow"),
    ("poem", "shark"),
    ("chocolate", "justice"),
    ("cathedral", "software"),
    ("gravity", "poet"),
    ("mystery", "cello"),
    ("statue", "gallop"),
    ("candle", "formula"),
    ("kitchen", "universe"),
    ("mirror", "whale"),
    ("piano", "cliff")
]

close_before = list()
close_after = list()
for w1, w2 in close_pairs:
    before, after = compute_similarity(w1, w2, vocab_emb_matrix, path, coefs, filter)
    close_before += [before]
    close_after += [after]

distant_before = list()
distant_after = list()
for w1, w2 in distant_pairs:
    before, after = compute_similarity(w1, w2, vocab_emb_matrix, path, coefs, filter)
    distant_before += [before]
    distant_after += [after]

print('')
print("Close pairs before:", np.mean(close_before))
print("Close pairs after:", np.mean(close_after))
print("Distant pairs before:", np.mean(distant_before))
print("Distant pairs after:", np.mean(distant_after))

# Save the results
df = pd.DataFrame({
    'word1': [w1 for w1, w2 in close_pairs + distant_pairs],
    'word2': [w2 for w1, w2 in close_pairs + distant_pairs],
    'before': [(v * 10000)//100 for v in close_before + distant_before],
    'after': [(v * 10000)//100 for v in close_after + distant_after],
})
df.to_csv('cosine_similarity_before_after.csv', index=False)

# closer : 
before = close_before + distant_before
after = close_after + distant_after
closer_before = [bef for i, bef in enumerate(before) if bef > after[i]] # Closer now, but value before
closer_after = [aft for i, aft in enumerate(after) if before[i] > aft] # Closer now, but value after
farer_before = [bef for i, bef in enumerate(before) if bef < after[i]] # Farer now, but value before
farer_after = [aft for i, aft in enumerate(after) if before[i] < aft] # Farer now, but value after

fig = go.Figure()
fig.add_trace(go.Line(x=closer_before, y=closer_after, mode='markers', name='Closer now'))
fig.add_trace(go.Line(x=farer_before, y=farer_after, mode='markers', name='Farer now'))
fig.add_trace(go.Line(x=[-1, 1], y=[-1, 1], mode='lines', name='y=x'))
fig.update_layout(title="Cosine similarity before and after", xaxis_title="Before", yaxis_title="After")
fig.show()

# Compute cosine similarity
# print("similary before", cosine_similarity(w1b, w2b))
# print("similary after", cosine_similarity(w1a, w2a))
# print("similary bef/aft", cosine_similarity(w1a, w1b))
# print("similary bef/aft", cosine_similarity(w2a, w2b))

joy happiness angry furious laugh giggle jump leap run sprint fear terror sad unhappy build construct breeze wind chilly cold tired exhausted story tale smile grin car automobile house home picture photograph child kid idea thought funny humorous quick rapid ocean sea weather climate mountain peak boat ship heart [UNK] mind intellect snake serpent flower blossom fire flame bird [UNK] star celestial river stream forest woods money currency glass crystal stone rock rain [UNK] thought idea danger [UNK] freedom liberty cat cement book apple moon chair water desert sun ice fish rocket music mountain computer river love equation light sadness art frog poetry insect happiness garbage science feather philosophy basketball thunder sorrow novel cactus ocean lantern painting hunger dream ladder galaxy ant pyramid novel keyboard volcano butterfly library castle equation symphony athlete rainforest algorithm theater molecule sculpture astronaut laptop meadow poem shark chocolate justice cathedral s

##### For words

In [80]:
def circular_convolution(signal, kernel):
    len_kernel = len(kernel)
    half_kernel = len_kernel // 2
    extended_signal = np.concatenate([signal[-half_kernel:], signal, signal[:half_kernel]]) # Étendre le signal aux deux extrémités pour gérer le cas circulaire
    result = np.convolve(extended_signal, kernel, mode='valid') # Appliquer la convolution standard
    return result

def compute_word_representation(word, vocab_emb_matrix, path, coefs, filter):
    """Compute the word representation after applying coefficient and convolution filter."""
    id = tokenizer.convert_tokens_to_ids([word])[0]
    print(str(tokenizer.decode([id])), end=' ')
    word_repr = np.array([vocab_emb_matrix[id, dim] * coefs[i] for i, dim in enumerate(path)])
    word_repr = circular_convolution(word_repr, filter)
    return word_repr

# Create the filter
N = 32
filter = [scipy.stats.norm.pdf(3*x/(N+1), 0, 1) for x in range(-N, N+1)]
# fig = px.line(filter)
# fig.show()

# List of words
words = ["kill", "death"]

# Compute representations for each word
word_representations = [compute_word_representation(word, vocab_emb_matrix, path, coefs, filter) for word in words]

# Plot the words
fig = go.Figure()

for word_repr, word in zip(word_representations, words):
    fig.add_trace(go.Scatterpolar(r=word_repr, name=word, fill='toself', opacity=0.50))

fig.update_layout(polar=dict(radialaxis=dict(range=[-3, 3])))
fig.show()


kill death 

##### For Sentences

In [1]:
def circular_convolution(signal, kernel):
    len_kernel = len(kernel)
    half_kernel = len_kernel // 2
    extended_signal = np.concatenate([signal[-half_kernel:], signal, signal[:half_kernel]]) # Étendre le signal aux deux extrémités pour gérer le cas circulaire
    result = np.convolve(extended_signal, kernel, mode='valid') # Appliquer la convolution standard
    return result
    
def compute_sentence_representation(text, path, coefs, filter):
    """Compute the word representation after applying coefficient and convolution filter."""
    tokens = tokenizer.tokenize(text)
    ids = tokenizer.convert_tokens_to_ids(tokens)
    #print(str(tokenizer.decode([id])), end=' ')
    emb = model(torch.tensor(ids).unsqueeze(0)).pooler_output[0].detach().numpy()
    word_repr = np.array([emb[dim] * coefs[i] for i, dim in enumerate(path)])
    word_repr = circular_convolution(word_repr, filter)
    return word_repr

# Create the filter
N = 0
filter = [scipy.stats.norm.pdf(3*x/(N+1), 0, 1) for x in range(-N, N+1)]
# fig = px.line(filter)
# fig.show()

# List of words
sentences = ["you", "hate you"]

# Compute representations for each word
sentences_repr = [compute_sentence_representation(s, path, coefs, filter) for s in sentences]

# Plot the words
fig = go.Figure()

for s_repr, s in zip(sentences_repr, sentences):
    fig.add_trace(go.Scatterpolar(r=s_repr, name=s, fill='toself', opacity=0.60))

fig.update_layout(polar=dict(radialaxis=dict(range=[-3, 3])))
fig.show()

NameError: name 'path' is not defined

##### Sentences but step by step

In [84]:
def circular_convolution(signal, kernel):
    len_kernel = len(kernel)
    half_kernel = len_kernel // 2
    extended_signal = np.concatenate([signal[-half_kernel:], signal, signal[:half_kernel]]) # Étendre le signal aux deux extrémités pour gérer le cas circulaire
    result = np.convolve(extended_signal, kernel, mode='valid') # Appliquer la convolution standard
    return result

def compute_sentence_representation(text, path, coefs, filter):
    """Compute the word representation after applying coefficient and convolution filter."""
    tokens = tokenizer.tokenize(text)
    ids = tokenizer.convert_tokens_to_ids(tokens)
    emb = model(torch.tensor(ids).unsqueeze(0)).pooler_output[0].detach().numpy()
    word_repr = np.array([emb[dim] * coefs[i] for i, dim in enumerate(path)])
    word_repr = circular_convolution(word_repr, filter)
    return word_repr


# Create the filter
N = 32
filter = [scipy.stats.norm.pdf(3*x/(N+1), 0, 1) for x in range(-N, N+1)]
# fig = px.line(filter)
# fig.show()

# Get the sentence
sentence = "The complex houses married and single soldiers and their families."
words = sentence.split()

# Compute representations for each partial sentence (from 1 word to full sentence)
partial_sentences = [" ".join(words[:i+1]) for i in range(len(words))]
representations = [compute_sentence_representation(s, path, coefs, filter) for s in partial_sentences]

# Create the base figure
fig = go.Figure()

# Add frames for each partial sentence
frames = [go.Frame(
    data=[go.Scatterpolar(r=rep, name=s, fill='toself', opacity=0.60)],
    name=s,
    layout=go.Layout(annotations=[
        dict(text=f"Sentence: {s}", showarrow=False, xref="paper", yref="paper", x=0, y=1.3, font=dict(size=14)),
        dict(text=f"Current word: {words[i]}", showarrow=False, xref="paper", yref="paper", x=0, y=1.2, font=dict(size=14, color="red"))
    ])
) for i, (rep, s) in enumerate(zip(representations, partial_sentences))]
fig.frames = frames

# Set up the animation settings
animation_settings = go.layout.Updatemenu(type="buttons", showactive=False, buttons=[dict(label="Play",
                                          method="animate", args=[None, dict(frame=dict(duration=1000, redraw=True), fromcurrent=True)])])

# Add the initial representation (of the first word) to the base figure
fig.add_trace(go.Scatterpolar(r=representations[0], name=partial_sentences[0], fill='toself', opacity=0.60))

# Add the initial annotations
fig.update_layout(
    annotations=[
        dict(text=f"Sentence: {partial_sentences[0]}", showarrow=False, xref="paper", yref="paper", x=0, y=1.3, font=dict(size=14)),
        dict(text=f"Current word: {words[0]}", showarrow=False, xref="paper", yref="paper", x=0, y=1.2, font=dict(size=14, color="red"))
    ],
    updatemenus=[animation_settings], 
    polar=dict(radialaxis=dict(range=[-3, 3]))
)

fig.show()


##### Save gif

In [85]:
# Save each frame as a png
for i, frame in enumerate(frames):
    frame_layout = frame.layout
    frame_layout.polar.radialaxis.range = [-3, 3]
    fig = go.Figure(data=frame.data, layout=frame_layout)
    fig.write_image(f"frames/frame{i}.png")

# Suppose your frames are saved as frame0.png, frame1.png, etc.
image_files = [f"frames/frame{i}.png" for i in range(len(frames))]

# Create the animation
images = [imageio.imread(image_file) for image_file in image_files]

# Save the animation
imageio.mimsave('imgs/animation2.gif', images, duration=1000, loop=0)  # 1 second per frame

# Delete all frames
for image_file in image_files:
    os.remove(image_file)

/var/folders/_8/z8rcf_ns4k9f2rw5zchj85mw0000gn/T/ipykernel_41006/4090931585.py:12: DeprecationWarning:

Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.

